    1) обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
    2) при обучении моделей обязательно использовать кроссвалидацию
    3) вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
    4) сделать выводы о том, какая модель справилась с задачей лучше других
    5) (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).
    
p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). Допустим, у нас две модели:

первая помечает 100 объектов как класс 1, но TP = 90
вторая помечает 1000 объектов как класс 1, но TP такой же - 90
Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

# Домашняя работа №3:

**Работа будет проводится на комплексном наборе данных для прогнозирования диабета на основе медицинских и демографических данных**:

Нам необходимо предсказать наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра. Датасет сформирован из 100.000 реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

**Объективные признаки:**

    Возраст
    Рост
    Вес
    Пол
    
**Результаты измерения:**

    Артериальное давление верхнее и нижнее
    Холестерин
    Глюкоза
    
**Субъективные признаки:**

    Курение
    Употребление Алкоголя
    Физическая активность
    
*Возраст дан в днях. Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.*

*Все показатели даны на момент осмотра.*

Скорее всего тут необходимо делать подготовку данных, проверить наличие пропусков (причем не обязательно это будет NaN (это как раз неплохо если он, его легче искать), могут поставить и 0 там где данных по клиенту не "хватило"(имеются в виду не бинарные признаки, а такие например как возраст) или просто написать "данных нет". вобщем вариантов пропусков может быть много), сделать балансировку классов, при необходимость, проверить наличие выбросов и т.д.

**Будем считать что работа по подготовке данных для последующего ML проведена и переходим к решению домашнего задания:**

In [1]:
#!pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, log_loss, confusion_matrix


%matplotlib inline

In [3]:
df = pd.read_csv('train_case2.csv', sep = ';' )
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [4]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', axis = 1), 
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [5]:
# Пайплайн для обработки данных, потом мы все трубочки сооденим в общий трубопровод
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [6]:
#Теперь объединим все наши трансформеры с помощью FeatureUnion:

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)

# FeatureUnion соединяет все наши отдельные трубочки(пайплайны) в 1 большую матицу которая будет поданна в качестве признака

feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [7]:
model = ['LogisticRegression', 'XGBClassifier', 'RandomForestClassifier']
fin_precision = []
fin_recall = []
fin_fscore = []
fin_roc_auc = []

In [8]:
#Добавим классификатор и запустим кросс-валидацию:

classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(C = 0.1, random_state = 42)),
])



#запустим кросс-валидацию
cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

fin_roc_auc.append(cv_score)

#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

CV score is 0.7847797116959196+-0.008728953249214647


In [9]:
#Посчитаем precision/recall/f_score

b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

fin_precision.append(precision[ix])
fin_recall.append(recall[ix])
fin_fscore.append(fscore[ix])

Best Threshold=0.389193, F-Score=0.729, Precision=0.645, Recall=0.839


In [10]:
#Добавим классификатор и запустим кросс-валидацию:

classifier_1 = Pipeline([
    ('features',feats),
    ('XGBClassifier', xgb.XGBClassifier(random_state=21)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_1, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

fin_roc_auc.append(cv_score)

#обучим пайплайн на всем тренировочном датасете
classifier_1.fit(X_train, y_train)
y_score_1 = classifier_1.predict_proba(X_test)[:, 1]

#model_xgb = xgb.XGBClassifier(random_state=21)

CV score is 0.7974879407860951+-0.006447877492510205


In [11]:
#Посчитаем precision/recall/f_score

b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_1)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

fin_precision.append(precision[ix])
fin_recall.append(recall[ix])
fin_fscore.append(fscore[ix])

Best Threshold=0.347103, F-Score=0.738, Precision=0.665, Recall=0.828


In [12]:
#Добавим классификатор и запустим кросс-валидацию:

classifier_2 = Pipeline([
    ('features',feats),
    ('RandomForestClassifier', RandomForestClassifier(random_state=21)),
])


#запустим кросс-валидацию
cv_scores = cross_val_score(classifier_2, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print('CV score is {}+-{}'.format(cv_score, cv_score_std))

fin_roc_auc.append(cv_score)

#обучим пайплайн на всем тренировочном датасете
classifier_2.fit(X_train, y_train)
y_score_2 = classifier_2.predict_proba(X_test)[:, 1]


CV score is 0.7740192922041457+-0.00753304357245972


In [13]:
#Посчитаем precision/recall/f_score

b=1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_2)
fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

fin_precision.append(precision[ix])
fin_recall.append(recall[ix])
fin_fscore.append(fscore[ix])

Best Threshold=0.340000, F-Score=0.721, Precision=0.641, Recall=0.825


In [14]:
results = pd.DataFrame(np.column_stack([model, fin_precision, fin_recall, fin_fscore, fin_roc_auc]), 
                               columns=['model', 'precision', 'recall', 'fscore', 'roc_auc'])

In [15]:
results

,model,precision,recall,fscore,roc_auc
0,LogisticRegression,0.6445073913428344,0.8388248847926267,0.7289382790208742,0.7847797116959196
1,XGBClassifier,0.6652479644707624,0.8283410138248848,0.7378899835796389,0.7974879407860951
2,RandomForestClassifier,0.6409383954154728,0.8246543778801844,0.7212817412333736,0.7740192922041457


Хуже всего справилась модель LogisticRegression, это может быть связано с тем, что из всех моделей она самая чувствительная(к выбросам, пропускам и т.д.) также она классическая, но старого поколения за счет чего может часто проигрывать более современным моделям даже написанным на ее же основе. RandomForestClassifier и XGBClassifier обе эти модели в основе имеют деревья и более хороший показатель XGBClassifier возможно связано с тем, что это более современная модель.

5) (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).

Кривые ROC следует использовать, когда имеется примерно одинаковое количество наблюдений для каждого класса.
Кривые Precision-Recall следует использовать при наличии дисбаланса класса от среднего до большого.
Причиной этой рекомендации является то, что кривые ROC представляют оптимистическую картину модели для наборов данных с классовым дисбалансом.

Однако кривые ROC могут представлять чрезмерно оптимистичное представление о производительности алгоритма, если в распределении классов имеется большой перекос. Кривые Precision-Recall (PR), были приведены в качестве альтернативы кривым ROC для задач с большим перекосом в распределении классов.

Соответственно если у нас данные примерно сбалансированные, то мы смело можем использовать roc_auc_curve, если мы имеем сильныйй перекос то необходимо использовать precision_recall